## Import deps

In [ ]:
!pip install tokun

%load_ext tensorboard

INFO: pip is looking at multiple versions of tokun to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.8 MB/s eta 0:00:00


In [ ]:
import datetime
import functools
import itertools
import math
import os
import urllib.request

import tensorflow as tf
import tensorflow_datasets as tfds

import mlable.tensorflow.io
import tokun.meta
import tokun.model
import tokun.pipeline

In [ ]:
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.15.0


## Setup the GPU / TPU

In [ ]:
tf.debugging.set_log_device_placement(False)

GPU = tf.config.list_logical_devices('GPU')
TPU = tf.config.list_logical_devices('TPU')

if TPU:
    RESOLVER = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(RESOLVER)
    tf.tpu.experimental.initialize_tpu_system(RESOLVER)
    DISTRIBUTION_STRATEGY = tf.distribute.TPUStrategy(RESOLVER)
elif GPU:
    DISTRIBUTION_STRATEGY = tf.distribute.MirroredStrategy(GPU)

print(DISTRIBUTION_STRATEGY)

## Defining The Metadata

In [ ]:
# PARAMETERS ##################################################################

ACTIVATION = 'silu'
ATTENTION = True
NORMALIZATION = True

N_TOKEN_DIM = [4, 4] # G, for each block

In [ ]:
# DERIVED #####################################################################

TOKEN_SIZES = list(itertools.accumulate(N_TOKEN_DIM, lambda x, y: x * y)) # in bytes

In [ ]:
# IMPORT ######################################################################

VERSION = tokun.meta.version(groups=N_TOKEN_DIM, activation=ACTIVATION, attention=ATTENTION, normalization=NORMALIZATION)
LABEL = '8.5'

URL_IMPORT = 'https://github.com/apehex/tokun/raw/main/models/{}/{}/{}/{}/{}.keras'.format(*VERSION, LABEL)
PATH_IMPORT = 'model.keras'

urllib.request.urlretrieve(URL_IMPORT, PATH_IMPORT)

('model.keras', <http.client.HTTPMessage at 0x7f7d087b9f90>)

In [ ]:
# SAMPLES #####################################################################

SAMPLES = [
    """A variational autoencoder is a generative model with a prior and noise distribution respectively. Usually such models are trained using the expectation-maximization meta-algorithm (e.g. probabilistic PCA, (spike & slab) sparse coding). Such a scheme optimizes a lower bound of the data likelihood, which is usually intractable, and in doing so requires the discovery of q-distributions, or variational posteriors. These q-distributions are normally parameterized for each individual data point in a separate optimization process. However, variational autoencoders use a neural network as an amortized approach to jointly optimize across data points. This neural network takes as input the data points themselves, and outputs parameters for the variational distribution. As it maps from a known input space to the low-dimensional latent space, it is called the encoder.""",
    """Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).""",
    """class AutoEncoder(tf.keras.models.Model):\n    def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n        super(AutoEncoder, self).__init__(**kwargs)\n        self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n        self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n    def call(self, x: tf.Tensor) -> tf.Tensor:\n        return self._decoder(self._encoder(x))""",
    """class AutoEncoder(tf.keras.models.Model):\n  def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n    super(AutoEncoder, self).__init__(**kwargs)\n    self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n    self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n  def call(self, x: tf.Tensor) -> tf.Tensor:\n    return self._decoder(self._encoder(x))""",
    """위키백과, 우리 모두의 백과사전.\nt-분포 확률적 임베딩(t-SNE)은 데이터의 차원 축소에 사용되는 기계 학습 알고리즘 중 하나로, 2002년 샘 로이스Sam Rowise와 제프리 힌튼에 의해 개발되었다.[1] t-SNE는 비선형 차원 축소 기법으로, 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용된다. 구체적으로 t-SNE는 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑한다.""",
    """Auto-encodeur variationnel\n\nArticle\nDiscussion\nLire\nModifier\nModifier le code\nVoir l’historique\n\nOutils\nEn apprentissage automatique, un auto-encodeur variationnel (ou VAE de l'anglais variational auto encoder)1, est une architecture de réseau de neurones artificiels introduite en 2013 par D. Kingma et M. Welling, appartenant aux familles des modèles graphiques probabilistes et des méthodes bayésiennes variationnelles.\n\nLes VAE sont souvent rapprochés des autoencodeurs2,3 en raison de leur architectures similaires. Leur utilisation et leur formulation mathématiques sont cependant différentes.\n\nLes auto-encodeurs variationnels permettent de formuler un problème d'inférence statistique (par exemple, déduire la valeur d'une variable aléatoire à partir d'une autre variable aléatoire) en un problème d'optimisation statistique (c'est-à-dire trouver les valeurs de paramètres qui minimisent une fonction objectif)4. Ils représentent une fonction associant à une valeur d'entrée une distribution latente multivariée, qui n'est pas directement observée mais déduite depuis un modèle mathématique à partir de la distribution d'autres variables. Bien que ce type de modèle ait été initialement conçu pour l'apprentissage non supervisé5, son efficacité a été prouvée pour l'apprentissage semi-supervisé6,7 et l'apprentissage supervisé8.\n\nArchitecture\nDans un VAE, les données d'entrée sont échantillonnées à partir d'une distribution paramétrée (la distribution a priori, en termes d'inférence bayésienne), et l'encodeur et le décodeur sont entraînés conjointement de sorte que la sortie minimise une erreur de reconstruction dans le sens de la divergence de Kullback-Leibler entre la distribution paramétrique postérieure et la vraie distribution a posteriori9,10.\n\nFormulation\n\nLe schéma de base d'un auto-encodeur variationnel. Le modèle reçoit \n𝑥\n{\displaystyle \mathbf {x} } comme entrée. L'encodeur le comprime dans l'espace latent. Le décodeur reçoit en entrée les informations prélevées dans l'espace latent et produit \n𝑥\n′\n{\displaystyle \mathbf {x'} } aussi semblable que possible à \n𝑥\n{\displaystyle \mathbf {x} } .\nOn note \n𝑥\n{\displaystyle \mathbf {x} } le vecteur contenant l'ensemble des variables observées que l'on souhaite modéliser. Ce vecteur est une variable aléatoire, caractérisé par une distribution de probabilité inconnue \n𝑃\n(\n𝑥\n)\n{\displaystyle P(\mathbf {x} )}, que l'on souhaite approximer par une distribution paramétrée \n𝑝\n𝜃{\displaystyle p_{\theta }} ayant pour paramètres \n𝜃{\displaystyle \theta }.\n\nOn introduit alors un vecteur aléatoire \n𝑧\n{\displaystyle \mathbf {z} } distribué conjointement avec \n𝑥\n{\displaystyle \mathbf {x} } (c'est-à-dire dont la loi de probabilité n'est pas indépendante de celle de \n𝑥\n{\displaystyle \mathbf {x} }). Ce vecteur \n𝑧\n{\displaystyle \mathbf {z} } représente un encodage latent de \n𝑥\n{\displaystyle \mathbf {x} }, que l'on ne peut observer directement.\n\nOn exprime alors la distribution \n𝑝\n𝜃{\displaystyle p_{\theta }} via la loi de probablitié marginale sur \n𝑧\n{\displaystyle \mathbf {z} }, ce qui donne alors:\n\n𝑝\n𝜃\n(\n𝑥\n)\n=\n∫\n𝑧\n𝑝\n𝜃\n(\n𝑥\n,\n𝑧\n)\n𝑑\n𝑧\n,\n{\displaystyle p_{\theta }(\mathbf {x} )=\int _{\mathbf {z} }p_{\theta }(\mathbf {x,z} )\,d\mathbf {z} ,}\noù \n𝑝\n𝜃\n(\n𝑥\n,\n𝑧\n)\n{\displaystyle p_{\theta }(\mathbf {x,z} )} représente la distribution conjointe sous \n𝑝\n𝜃{\displaystyle p_{\theta }} des données observables \n𝑥\n{\displaystyle \mathbf {x} } et de leur représentation latente \n𝑧\n{\displaystyle \mathbf {z} }. Selon la formule des probabilités composées, l'équation peut être réécrite comme\n\n𝑝\n𝜃\n(\n𝑥\n)\n=\n∫\n𝑧\n𝑝\n𝜃\n(\n𝑥\n∣\n𝑧\n)\n𝑝\n𝜃\n(\n𝑧\n)\n𝑑\n𝑧\n{\displaystyle p_{\theta }(\mathbf {x} )=\int _{\mathbf {z} }p_{\theta }(\mathbf {x\mid z} )p_{\theta }(\mathbf {z} )\,d\mathbf {z} }\nDans l'auto-encodeur variationnel classique, on fait l'hypothèse que \n𝑧\n{\displaystyle \mathbf {z} } est un vecteur à valeur réelles de dimension finie, et \n𝑝\n𝜃\n(\n𝑥\n|\n𝑧\n)\n{\displaystyle p_{\theta }(\mathbf {x|z} )} suit une loi normale. Par conséquent, \n𝑝\n𝜃\n(\n𝑥\n)\n{\displaystyle p_{\theta }(\mathbf {x} )} est un mélange de distributions gaussiennes.\n\nOn peut voir les relations entre les données d'entrée et leur représentation latente comme un problème d'inférence bayésienne avec\n\n𝑝\n𝜃\n(\n𝑧\n)\n{\displaystyle p_{\theta }(\mathbf {z} )} représente la distribution probabilité a priori dans l'espace latent\n𝑝\n𝜃\n(\n𝑥\n∣\n𝑧\n)\n{\displaystyle p_{\theta }(\mathbf {x} \mid \mathbf {z} )} représente la vraisemblance\n𝑝\n𝜃\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle p_{\theta }(\mathbf {z} \mid \mathbf {x} )} représente la distribution de probabilité a posteriori\nMalheureusement, le calcul de \n𝑝\n𝜃\n(\n𝑥\n)\n{\displaystyle p_{\theta }(\mathbf {x} )} est au mieux coûteux, et dans la plupart des cas, impossible. Pour résoudre ce problème, il est nécessaire d'introduire une autre fonction \n𝑞\nΦ{\displaystyle q_{\Phi }} pour approximer la distribution a posteriori :\n\n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n≈\n𝑝\n𝜃\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle q_{\Phi }(\mathbf {z\mid x} )\approx p_{\theta }(\mathbf {z\mid x} )}\noù \nΦ{\displaystyle \Phi } est l'ensemble des paramètres de \n𝑞\n{\displaystyle q} .\n\nAinsi le problème est formulé pour pouvoir être appliqué dans une architecture de réseau de neurones auto-encodeur, dans lequel la distribution de vraisemblance conditionnelle \n𝑝\n𝜃\n(\n𝑥\n∣\n𝑧\n)\n{\displaystyle p_{\theta }(\mathbf {x} \mid \mathbf {z} )} est représentée par un décodeur probabiliste, tandis que la distribution a posteriori approchée \n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle q_{\Phi }(\mathbf {z} \mid \mathbf {x} )} est représentée par un codeur probabiliste. La mise en œuvre d'un VAE consistera donc à calculer les valeurs optimales des paramètres \n𝜃{\displaystyle \theta } et \nΦ{\displaystyle \Phi } par un apprentissage automatique.\n\nFonction de perte ELBO\nComme dans tout problème d'apprentissage profond, il est nécessaire de définir une fonction de perte différentiable afin de mettre à jour les poids du réseau par rétropropagation lors de l'apprentissage.\n\nPour les auto-encodeurs variationnels, l'idée est de minimiser conjointement les paramètres du modèle génératif \n𝜃{\displaystyle \theta } pour réduire l'erreur de reconstruction entre l'entrée et la sortie, et \nΦ{\displaystyle \Phi } pour avoir \n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle q_{\Phi }(\mathbf {z\mid x} )}, la distribution postérieure approchée, le plus près possible de \n𝑝\n𝜃\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle p_{\theta }(\mathbf {z} \mid \mathbf {x} )}, la vraie distribution de probabilité a posteriori.\n\nComme fonction de coût pour la reconstruction, l'erreur quadratique moyenne et l'entropie croisée sont souvent utilisées.\n\nPour la fonction de coût de distance entre les deux distributions, la divergence inverse de Kullback – Leibler \n𝐷\n𝐾\n𝐿\n(\n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n∥\n𝑝\n𝜃\n(\n𝑧\n∣\n𝑥\n)\n)\n{\displaystyle D_{KL}(q_{\Phi }(\mathbf {z\mid x} )\parallel p_{\theta }(\mathbf {z\mid x} ))} est un bon choix pour pousser \n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle q_{\Phi }(\mathbf {z\mid x} )} en dessous de \n𝑝\n𝜃\n(\n𝑧\n∣\n𝑥\n)\n{\displaystyle p_{\theta }(\mathbf {z} \mid \mathbf {x} )}1,11.\n\nReparamétrisation\n\nLe schéma de l'astuce de reparamétrisation. La variable aléatoire \n𝜀{\displaystyle \mathbf {\varepsilon } } est injecté dans l'espace latent, non observé, \n𝑧\n{\displaystyle \mathbf {z} } comme entrée externe. De cette manière, il est possible de rétropropager le gradient sans impliquer de variable stochastique lors de la mise à jour.\nPour rendre la formulation ELBO adaptée à des fins d'apprentissage, il est nécessaire de modifier légèrement la formulation du problème et la structure du VAE12.\n\nL'échantillonnage stochastique est l'opération non différentiable par laquelle il est possible d'échantillonner à partir de l'espace latent et d'alimenter le décodeur probabiliste.\n\nL'hypothèse principale sur l'espace latent est qu'il peut être considéré comme un ensemble de distributions gaussiennes multivariées, et peut donc être décrit comme\n\n𝑧\n∼\n𝑞\nΦ\n(\n𝑧\n∣\n𝑥\n)\n=\n𝑁\n(\n𝜇\n,\n𝜎\n2\n)\n{\displaystyle \mathbf {z} \sim q_{\Phi }(\mathbf {z} \mid \mathbf {x} )={\mathcal {N}}({\boldsymbol {\mu }},{\boldsymbol {\sigma }}^{2})} .\n\nLe schéma d'un auto-encodeur variationnel après l'astuce de reparamétrisation.\nEtant donné \n𝜀\n∼\n𝑁\n(\n0\n,\n𝐼\n)\n{\displaystyle {\boldsymbol {\varepsilon }}\sim {\mathcal {N}}(0,{\boldsymbol {I}})} et \n⊙{\displaystyle \odot } défini comme le produit élément par élément, l'astuce de reparamétrisation modifie l'équation ci-dessus comme\n\n𝑧\n=\n𝜇\n+\n𝜎\n⊙\n𝜀\n.\n{\displaystyle \mathbf {z} ={\boldsymbol {\mu }}+{\boldsymbol {\sigma }}\odot {\boldsymbol {\varepsilon }}.}\nGrâce à cette transformation (qui peut être étendue à des distributions non gaussiennes), le VAE devient entraînable et le codeur probabiliste doit apprendre à mapper une représentation compressée de l'entrée dans les deux vecteurs latents \n𝜇{\displaystyle {\boldsymbol {\mu }}} et \n𝜎{\displaystyle {\boldsymbol {\sigma }}}, tandis que la stochasticité reste exclue du processus de mise à jour et est injectée dans l'espace latent en tant qu'entrée externe via le vecteur aléatoire \n𝜀{\displaystyle {\boldsymbol {\varepsilon }}} .\n\nRéférences\nDiederik P. Kingma et Max Welling, « Auto-Encoding Variational Bayes », arXiv:1312.6114 [cs, stat],\x200e 1er mai 2014 (lire en ligne [archive], consulté le 1er juillet 2022)\n(en) Kramer, « Nonlinear principal component analysis using autoassociative neural networks », AIChE Journal, vol. 37, no 2,\x200e 1991, p. 233–243 (DOI 10.1002/aic.690370209, lire en ligne [archive])\n(en) Hinton et Salakhutdinov, « Reducing the Dimensionality of Data with Neural Networks », Science, vol. 313, no 5786,\x200e 28 juillet 2006, p. 504–507 (PMID 16873662, DOI 10.1126/science.1127647, Bibcode 2006Sci...313..504H, S2CID 1658773, lire en ligne [archive])\n(en) « A Beginner's Guide to Variational Methods: Mean-Field Approximation [archive] », Eric Jang, 8 juillet 2016\nWei-Ning Hsu, Yu Zhang et James Glass, 2017 IEEE Automatic Speech Recognition and Understanding Workshop (ASRU), décembre 2017, 16–23 p. (ISBN 978-1-5090-4788-8, DOI 10.1109/ASRU.2017.8268911, arXiv 1707.06265, S2CID 22681625), « Unsupervised domain adaptation for robust speech recognition via variational autoencoder-based data augmentation »\nM. Ehsan Abbasnejad, Anthony Dick et Anton van den Hengel, Infinite Variational Autoencoder for Semi-Supervised Learning, 2017, 5888–5897 p. (lire en ligne [archive])\n(en) Xu, Sun, Deng et Tan, « Variational Autoencoder for Semi-Supervised Text Classification », Proceedings of the AAAI Conference on Artificial Intelligence, vol. 31, no 1,\x200e 12 février 2017 (lire en ligne [archive])\nKameoka, Li, Inoue et Makino, « Supervised Determined Source Separation with Multichannel Variational Autoencoder », Neural Computation, vol. 31, no 9,\x200e 1er septembre 2019, p. 1891–1914 (PMID 31335290, DOI 10.1162/neco_a_01217, S2CID 198168155, lire en ligne [archive])\nAn, J., & Cho, S. (2015). Variational autoencoder based anomaly detection using reconstruction probability. Special Lecture on IE, 2(1).\nKingma et Welling, « An Introduction to Variational Autoencoders », Foundations and Trends in Machine Learning, vol. 12, no 4,\x200e 2019, p. 307–392 (ISSN 1935-8237, DOI 10.1561/2200000056, arXiv 1906.02691, S2CID 174802445)\n(en) « From Autoencoder to Beta-VAE [archive] », Lil'Log, 12 août 2018\nBengio, Courville et Vincent, « Representation Learning: A Review and New Perspectives », IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 35, no 8,\x200e 2013, p. 1798–1828 (ISSN 1939-3539, PMID 23787338, DOI 10.1109/TPAMI.2013.50, arXiv 1206.5538, S2CID 393948, lire en ligne [archive])""",]

## Init

In [ ]:
# INIT ########################################################################

with DISTRIBUTION_STRATEGY.scope():
    MODEL = tf.keras.models.load_model(PATH_IMPORT)
    MODEL.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0., axis=-1, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE, name='loss'),
        metrics=['accuracy'])


In [ ]:
MODEL.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  593408    
                                                                 
 decoder (Decoder)           multiple                  595200    
                                                                 
Total params: 1188608 (4.53 MB)
Trainable params: 1188608 (4.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Tokenize

In [ ]:
# ENCODE ######################################################################

__s = SAMPLES[1]
__x = tokun.pipeline.preprocess(text=__s, groups=N_TOKEN_DIM, flatten=True)
__e = MODEL._encoder(__x) # final embedding = input for another model

In [ ]:
# VIEW ########################################################################

print(len(__s)) # original text length, roughly S
print(__x.shape) # 4 * S, with padding
print(__e.shape) # 4 * S // 64

134
(544, 256)
(34, 256)


## Detokenize

In [ ]:
# DECODE ######################################################################

__p = MODEL._decoder(__e)
__y = tokun.pipeline.postprocess(__p)

In [ ]:
print('# INPUT ################################################################\n\n' + __s)
print('\n# OUTPUT ###############################################################\n\n' + __y)
print('\n# SCORE ################################################################\n\n' + str(tokun.pipeline.compare(__s, __y)))

# INPUT ################################################################

Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).

# OUTPUT ###############################################################

Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).  

# SCORE ################################################################

1.0


## Robustness

In [ ]:
# SAMPLE ######################################################################

__x = tokun.pipeline.preprocess("""Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).""", groups=N_TOKEN_DIM, flatten=True)
__e = MODEL._encoder(__x)

In [ ]:
# NOISE #######################################################################

__std = tf.math.reduce_std(__e, axis=0)
__noise = tf.random.normal(shape=(256,), mean=0., stddev=tf.math.reduce_mean(__std).numpy())

In [ ]:
# DETOKENIZE ##################################################################

print(tokun.pipeline.postprocess(MODEL._decoder(__e))) # original
print(tokun.pipeline.postprocess(MODEL._decoder(__e + 0.8 * __std))) # with structured noise
print(tokun.pipeline.postprocess(MODEL._decoder(__e + 1. * __noise))) # with random noise

Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).  
Une unité lexicale ou token lexical ou plus simple ent token est un couple composé d'un nom et d'une valeur optionnelle (e.g. 135677).  
Une unité lexicale ou token lexical ou plus simplement token est un couple composé d'un nom et d'une vaɬeur optionnelle (e.g. 135677). e


In [ ]:
# ENCODE ######################################################################

__s = SAMPLES[0]
__x = tokun.pipeline.preprocess(text=__s, groups=N_TOKEN_DIM, flatten=True)
__e = MODEL._encoder(__x) # final embedding = input for another model

In [ ]:
# DECODE ######################################################################

__p = MODEL._decoder(__e)
__y = tokun.pipeline.postprocess(__p)

In [ ]:
print(__s)
print(__y)
print(tokun.pipeline.compare(__s, __y))

Auto-encodeur variationnel

Article
Discussion
Lire
Modifier
Modifier le code
Voir l’historique

Outils
En apprentissage automatique, un auto-encodeur variationnel (ou VAE de l'anglais variational auto encoder)1, est une architecture de réseau de neurones artificiels introduite en 2013 par D. Kingma et M. Welling, appartenant aux familles des modèles graphiques probabilistes et des méthodes bayésiennes variationnelles.

Les VAE sont souvent rapprochés des autoencodeurs2,3 en raison de leur architectures similaires. Leur utilisation et leur formulation mathématiques sont cependant différentes.

Les auto-encodeurs variationnels permettent de formuler un problème d'inférence statistique (par exemple, déduire la valeur d'une variable aléatoire à partir d'une autre variable aléatoire) en un problème d'optimisation statistique (c'est-à-dire trouver les valeurs de paramètres qui minimisent une fonction objectif)4. Ils représentent une fonction associant à une valeur d'entrée une distribution 

In [ ]:
print(len(__s))
print(__e.shape)
tf.print(__e[:4, :8], summarize=-1)

134
(34, 256)
[[3.17276168 1.53056908 2.41119337 0.0258403085 1.5207386 1.66698301 2.24263883 2.11223722]
 [2.65205669 1.68546355 2.01416564 0.655108571 2.3957293 1.70228446 2.12328672 2.04205203]
 [2.4943645 0.441500723 1.79073346 2.31724644 1.87132716 1.36434507 3.37104845 2.3522613]
 [2.87078524 1.11898732 2.12827492 0.995271683 0.403087556 0.974042118 1.82035911 2.90426946]]


In [ ]:
__t = list(set(tokun.pipeline.chunk(seq=SAMPLES[-1], size=4, repeats=False)))
__x = tokun.pipeline.preprocess(text=''.join(__t), groups=N_TOKEN_DIM, flatten=True)
__e = MODEL._encoder(__x)

In [ ]:
mlable.io.write(data=__t, path='./metadata.tsv', tsv=False)
mlable.io.write(data=__e.numpy(), path='./embeddings.tsv', tsv=True)